In [1]:
# 基本パッケージ（numpy,Pandas,matplotlib）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
# LightGBM
import lightgbm as lgb
# XGboost
import xgboost as xgb
# train_test_split（データを分割出してくれる）
from sklearn.model_selection import train_test_split
# accuracy_score（正解率を測れる）
from sklearn.metrics import accuracy_score
# グリッドサーチ（ハイパーパラメータを自動的に最適化してくれる）
from sklearn.model_selection import GridSearchCV
# 正規化
from sklearn.preprocessing import MinMaxScaler
# 交差検証
from sklearn.model_selection import cross_val_score
# 特徴量選択用
from sklearn.feature_selection import SelectFromModel
# warningの抑制
import warnings
# モデルの保存
from sklearn.externals import joblib

D:\Program\Anaconda3\envs\sklearn1.9.1\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')

In [3]:
# 株価データの読み込み
stock_data = pd.read_csv("stock_Price_Prediction_v3.2.csv", encoding="shift-jis")

# 読み込みデータの表示
stock_data.head()

,USD,GBP,EUR,CAD,CHF,SEK,DKK,NOK,AUD,NZD,ZAR,BHD,IDR(100),CNY,HKD,INR,PHP,SGD,KRW(100),THB,KWD,SAR,AED,MXN,TWD,Open000001.SS,High000001.SS,Low000001.SS,Adj Close000001.SS,Open^AORD,High^AORD,Low^AORD,Open^AXJO,High^AXJO,Low^AXJO,Adj Close^AXJO,Open^BFX,High^BFX,Low^BFX,Adj Close^BFX,Open^BSESN,High^BSESN,Low^BSESN,Adj Close^BSESN,Open^BVSP,High^BVSP,Low^BVSP,Adj Close^BVSP,Open^DJI,High^DJI,Low^DJI,Adj Close^DJI,Open^FCHI,High^FCHI,Low^FCHI,Adj Close^FCHI,Open^GDAXI,High^GDAXI,Low^GDAXI,Adj Close^GDAXI,Open^GSPC,High^GSPC,Low^GSPC,Open^GSPTSE,High^GSPTSE,Low^GSPTSE,Adj Close^GSPTSE,Open^HSI,High^HSI,Low^HSI,Adj Close^HSI,Open^IPSA,High^IPSA,Low^IPSA,Adj Close^IPSA,Open^IXIC,High^IXIC,Low^IXIC,Adj Close^IXIC,Open^JKSE,High^JKSE,Low^JKSE,Adj Close^JKSE,Open^KS11,High^KS11,Low^KS11,Open^MERV,High^MERV,Low^MERV,Adj Close^MERV,Open^MXX,High^MXX,Low^MXX,Adj Close^MXX,Open^N100,High^N100,Low^N100,Adj Close^N100,Open^NYA,High^NYA,Low^NYA,Adj Close^NYA,Open^RUT,High^RUT,Low^RUT,Adj Close^RUT,High^TWII,Low^TWII,Adj Close^TWII,Open^VIX,High^VIX,Adj Close^VIX,Open^XAX,High^XAX,Low^XAX,Adj Close^XAX,Open,High,Low,Volume,Adj Close,answer,Low^VIX,Open^TWII,Adj Close^GSPC,Adj Close^KS11,Adj Close^AORD,Low^VIX^2,Low^VIX Open^TWII,Low^VIX Adj Close^GSPC,Low^VIX Adj Close^KS11,Low^VIX Adj Close^AORD,Open^TWII^2,Open^TWII Adj Close^GSPC,Open^TWII Adj Close^KS11,Open^TWII Adj Close^AORD,Adj Close^GSPC^2,Adj Close^GSPC Adj Close^KS11,Adj Close^GSPC Adj Close^AORD,Adj Close^KS11^2,Adj Close^KS11 Adj Close^AORD,Adj Close^AORD^2
0,0.000000,-0.002711,-0.001877,0.003238,-0.001622,-0.003846,-0.001268,-0.000656,0.001690,-0.000849,0.009338,0.000000,-0.007246,0.000000,0.000000,0.003663,0.000000,-0.000831,-0.008893,-0.006536,0.003609,0.000000,0.000000,-0.002027,0.000000,-0.016024,-0.004302,0.004251,0.007344,-0.007255,-0.006006,-0.001020,-0.008053,-0.007226,-0.000710,-0.002805,-0.004067,0.006438,0.002904,0.010276,-0.001324,-0.010327,-0.009173,-0.012208,-0.014983,-0.014038,-0.013892,-0.012080,-0.003927,-0.001217,-0.012214,-0.011191,-0.011637,-0.009375,-0.009456,0.001065,-0.016036,-0.010612,-0.004488,-0.005506,-0.008530,-0.006707,-0.014114,-0.003407,-0.004167,-0.006936,-0.006474,-0.018231,-0.018280,-0.003118,-0.004052,-0.009256,-0.003805,-0.000105,0.005605,-0.014401,-0.014141,-0.018726,-0.011112,0.003838,0.000590,0.004085,-0.001906,0.023984,0.016071,0.030788,-0.002590,-0.027515,-0.014697,-0.008481,-0.007267,-0.009166,-0.017076,-0.017050,-0.006375,-0.007674,-0.007250,-0.001550,-0.009459,-0.009459,-0.009459,-0.009459,-0.007948,-0.005887,-0.009231,-0.007772,-0.003112,-0.003727,0.008201,0.015167,0.054611,0.054280,0.001202,-0.003871,-0.007277,-0.008593,-0.006250,0.024889,-0.003604,2.643314,0.018700,1.0,-0.034943,0.000000,-0.003205,0.000000,0.005206,0.001221,-0.000000,0.000112,-0.000000,-0.000182,0.000000,-0.000000,0.000000,0.000000e+00,0.000010,-0.000000,-1.668653e-05,0.000000,0.000000e+00,2.710567e-05
1,-0.000751,-0.000680,0.001624,0.000837,0.002000,-0.001544,0.001270,0.006566,-0.004639,-0.004931,0.010934,-0.000735,0.000000,-0.000622,-0.001171,-0.003650,-0.003817,0.001941,-0.000997,0.003289,-0.003367,-0.000844,-0.000827,-0.001354,0.000000,0.007755,0.024780,0.000608,0.029596,-0.002965,-0.004128,-0.003605,-0.002805,-0.004479,-0.003582,-0.001539,0.007597,0.008347,0.009030,0.007857,-0.010703,0.008073,0.001284,0.014311,-0.014307,0.001873,-0.001531,0.021016,-0.010887,-0.007542,0.001753,0.003616,-0.002981,-0.004875,-0.010073,-0.020282,-0.001732,-0.006093,-0.010315,-0.005091,-0.009993,-0.007376,0.000339,-0.004867,-0.003182,-0.007444,-0.007370,-0.005971,-0.001443,-0.003247,-0.000239,0.006228,0.006706,0.006440,0.003402,-0.018269,-0.006910,-0.000254,0.003026,0.001560,0.021175,0.003847,0.029382,0.014400,0.001065,0.011734,-0.004941,0.004482,-0.026619,-0.037923,-0.017218,-0.008022,-0.003079,0.008560,-0.005135,-0.005065,-0.012784,-0.016449,0.000559,0.000559,0.000559,0.000559,-0.007772,-0.005882,-0.002279,0.003564,0.000140,-0.006939,-0.013912,0.025245,-0.018564,-0.020792,-0.008593

In [4]:
# DataFrameをNumPy配列へ変換
# 正解ラベルの変換
answers = stock_data.answer.values
# 説明変数の変換
stock_data.drop(columns=['answer'], inplace=True)
explanatory_variable = stock_data.values

In [5]:
# スケール変換       
ms = MinMaxScaler()
ms.fit(explanatory_variable)
explanatory_variable = ms.transform(explanatory_variable)

In [6]:
# データの分割（データの80%を訓練用に、20％をテスト用に分割する）
X_train, X_test, y_train, y_test = train_test_split(explanatory_variable , answers, test_size=0.2, random_state=1,shuffle = False)

In [7]:
# グリッドサーチするパラメータを設定
parameters = {'n_estimators':[1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

# グリッドサーチを実行
rf = joblib.load('rf.pkl');
grid_search = GridSearchCV(rf, param_grid=parameters, cv=10)
grid_search = grid_search.fit(X_train , y_train)
 
# グリッドサーチ結果(最適パラメータ)を取得
best_params = grid_search.best_params_
print ("最適パラメータ：{}".format(grid_search.best_params_))

最適パラメータ：{'n_estimators': 60}


In [8]:
# 最適パラメーターを指定して学習
clf = RandomForestClassifier(n_estimators=best_params['n_estimators'], random_state=1)
clf.fit(X_train , y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [9]:
# 学習後のモデルによるテスト
# トレーニングデータを用いた予測
y_train_pred = clf.predict(X_train)
# テストデータを用いた予測
y_val_pred = clf.predict(X_test)

In [10]:
# 正解率の計算
train_score = clf.score(X_train , y_train)
test_score = clf.score(X_test, y_test)
# 正解率を表示
print("トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("テストデータに対する正解率：" + str(test_score * 100) + "%")

トレーニングデータに対する正解率：100.0%
テストデータに対する正解率：56.27329192546584%


In [11]:
#  交差検証
## 5分割し交差検証
scores = cross_val_score(clf, explanatory_variable, answers, cv=5)
## 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
## スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

Cross-Validation scores: [0.57888199 0.59378882 0.53913043 0.55666252 0.56164384]
Average score: 0.5660215186838177


In [12]:
# モデルベース特徴量選択による特徴量選択
# estimatorとしてランダムフォレストを使用。重要度がmedian（中央値）以上のものを選択
selector_RF = SelectFromModel(RandomForestClassifier(random_state=1), threshold="median")
selector_RF.fit(X_train , y_train)

# 選択された特徴量による学習：モデルベース特徴量選択
# 選択された特徴量のみにデータを変換
X_train_RF = selector_RF.transform(X_train)
X_test_RF = selector_RF.transform(X_test)

# 学習
clf.fit(X_train_RF, y_train)

# 正解率の算出
y_train_pred_RF = clf.predict(X_train_RF)
y_val_pred_RF = clf.predict(X_test_RF)
train_score = accuracy_score(y_train, y_train_pred_RF)
test_score = accuracy_score(y_test, y_val_pred_RF)

print("モデルベース特徴量選択(RandomForest)：トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("モデルベース特徴量選択(RandomForest)：テストデータに対する正解率：" + str(test_score * 100) + "%")

#  交差検証
## 選択された特徴量のみにデータを変換
explanatory_variable_RF = selector_RF.transform(explanatory_variable)
## 5分割し交差検証
scores = cross_val_score(clf, explanatory_variable_RF, answers, cv=5)
## 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
## スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

モデルベース特徴量選択(RandomForest)：トレーニングデータに対する正解率：100.0%
モデルベース特徴量選択(RandomForest)：テストデータに対する正解率：59.25465838509317%
Cross-Validation scores: [0.58757764 0.60248447 0.5826087  0.57160648 0.58530511]
Average score: 0.5859164778045064


In [13]:
# モデルベース特徴量選択による特徴量選択
# estimatorとしてLightGBMを使用。重要度がmedian（中央値）以上のものを選択
selector_LGBM = SelectFromModel(lgb.LGBMClassifier(random_state=1), threshold="median")
selector_LGBM.fit(X_train , y_train)

# 選択された特徴量による学習：モデルベース特徴量選択
# 選択された特徴量のみにデータを変換
X_train_LGBM = selector_LGBM.transform(X_train)
X_test_LGBM = selector_LGBM.transform(X_test)

# 学習
clf.fit(X_train_LGBM, y_train)

# 正解率の算出
y_train_pred_LGBM = clf.predict(X_train_LGBM)
y_val_pred_LGBM = clf.predict(X_test_LGBM)
train_score = accuracy_score(y_train, y_train_pred_LGBM)
test_score = accuracy_score(y_test, y_val_pred_LGBM)

print("モデルベース特徴量選択(LightGBM)：トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("モデルベース特徴量選択(LightGBM)：テストデータに対する正解率：" + str(test_score * 100) + "%")

#  交差検証
## 選択された特徴量のみにデータを変換
explanatory_variable_LGBM = selector_LGBM.transform(explanatory_variable)
## 5分割し交差検証
scores = cross_val_score(clf, explanatory_variable_LGBM, answers, cv=5)
## 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
## スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

モデルベース特徴量選択(LightGBM)：トレーニングデータに対する正解率：100.0%
モデルベース特徴量選択(LightGBM)：テストデータに対する正解率：58.01242236024845%
Cross-Validation scores: [0.58012422 0.56770186 0.58385093 0.60772105 0.57285181]
Average score: 0.5824499740878537


In [14]:
# モデルベース特徴量選択による特徴量選択
# estimatorとしてXGboostを使用。重要度がmedian（中央値）以上のものを選択
selector_XGB = SelectFromModel(xgb.XGBClassifier(random_state=1), threshold="median")
selector_XGB.fit(X_train , y_train)

# 選択された特徴量による学習：モデルベース特徴量選択
# 選択された特徴量のみにデータを変換
X_train_XGB = selector_XGB.transform(X_train)
X_test_XGB = selector_XGB.transform(X_test)

# 学習
clf.fit(X_train_XGB, y_train)

# 正解率の算出
y_train_pred_XGB = clf.predict(X_train_XGB)
y_val_pred_XGB = clf.predict(X_test_XGB)
train_score = accuracy_score(y_train, y_train_pred_XGB)
test_score = accuracy_score(y_test, y_val_pred_XGB)

print("モデルベース特徴量選択(XGboost)：トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("モデルベース特徴量選択(XGboost)：テストデータに対する正解率：" + str(test_score * 100) + "%")

#  交差検証
## 選択された特徴量のみにデータを変換
explanatory_variable_XGB = selector_XGB.transform(explanatory_variable)
## 5分割し交差検証
scores = cross_val_score(clf, explanatory_variable_XGB, answers, cv=5)
## 各分割におけるスコア
print('Cross-Validation scores: {}'.format(scores))
## スコアの平均値
print('Average score: {}'.format(np.mean(scores)))

モデルベース特徴量選択(XGboost)：トレーニングデータに対する正解率：100.0%
モデルベース特徴量選択(XGboost)：テストデータに対する正解率：57.391304347826086%
Cross-Validation scores: [0.58509317 0.59378882 0.56397516 0.57036115 0.58530511]
Average score: 0.5797046788827611
